### Preparing VAERS data of all years (2020-2025)

In [2]:
# importing libraries
import pandas as pd
import os
import glob
from IPython.display import display, HTML

In [4]:
# Fucntion to load data for all years (2020 - 2025) and join all tables

import os
import pandas as pd

def clean_and_join_vaers(year, base_path):
    """
    Clean and join VAERS DATA + VAX + SYMPTOMS for years 2020 to 2025
    
    """

    # -----------------------------
    # File paths
    # -----------------------------
    data_file = os.path.join(base_path, f"{year}VAERSDATA.csv")
    vax_file = os.path.join(base_path, f"{year}VAERSVAX.csv")
    symptoms_file = os.path.join(base_path, f"{year}VAERSSYMPTOMS.csv")

    # -----------------------------
    # 1. Load data
    # -----------------------------
    df_data = pd.read_csv(data_file, encoding="ISO-8859-1")
    df_vax = pd.read_csv(vax_file, encoding="ISO-8859-1")
    df_sym = pd.read_csv(symptoms_file, encoding="ISO-8859-1")


    # -----------------------------
    # 2. Clean & transform symptoms
    # -----------------------------
    # Drop SYMPTOMVERSION cols
    df_sym = df_sym.loc[:, ~df_sym.columns.str.startswith('SYMPTOMVERSION')]

    # Sort by VAERS_ID and ORDER
    df_sym = df_sym.sort_values(by=['VAERS_ID', 'ORDER']).reset_index(drop=True)

    # List of SYMPTOM columns
    symptom_cols = [col for col in df_sym.columns if col.startswith('SYMPTOM')]

    # Create ALL_SYMPTOMS as list per row
    df_sym['ALL_SYMPTOMS'] = df_sym[symptom_cols].apply(
        lambda x: [sym for sym in x if pd.notna(sym)], axis=1
    )

    # Drop only the original SYMPTOM columns and keep 'ORDER' column 
    df_sym.drop(symptom_cols, axis=1, inplace=True)

    # Group by VAERS_ID + ORDER
    df_sym = df_sym.groupby(['VAERS_ID', 'ORDER'])['ALL_SYMPTOMS'] \
        .apply(lambda lists: [sym for sublist in lists for sym in sublist]) \
        .reset_index()

    # -----------------------------
    # 3. Filter VAX only for COVID
    # -----------------------------
    df_vax_covid = df_vax[df_vax['VAX_TYPE'].str.startswith('COVID')].copy()

    # Sort by VAERS_ID + ORDER
    df_vax_covid = df_vax_covid.sort_values(
        by=['VAERS_ID', 'ORDER']
    ).reset_index(drop=True)

    # -----------------------------
    # 4. Join VAX + SYMPTOMS
    # -----------------------------
    covid_records = df_vax_covid.merge(df_sym, on=['VAERS_ID', 'ORDER'], how='inner')

    # -----------------------------
    # 5. Join DATA + (VAX+SYMPTOMS)
    # -----------------------------
    final = df_data.merge(covid_records, on=['VAERS_ID', 'ORDER'], how='inner')

    # Track year
    final['YEAR'] = year

    return final




In [5]:
base_dir = "/Users/joeraymond/Desktop/Data_Science_Projects/ADEGuard_AI_System/data/raw"

# Find all year folders
year_folders = sorted(glob.glob(os.path.join(base_dir, "*VAERSData")))

all_years = []

for folder in year_folders:
    year = int(os.path.basename(folder)[:4])  # e.g. "2020VAERSData" -> 2020
    print(f"Processing {year} ...")
    merged = clean_and_join_vaers(year, folder)
    all_years.append(merged)

# Final combined dataset
vaers_all = pd.concat(all_years, ignore_index=True)


Processing 2020 ...


/var/folders/cr/048sbrkd4k5f7y4yw3rstw7m0000gn/T/ipykernel_52998/3014866597.py:22: DtypeWarning: Columns (12,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_data = pd.read_csv(data_file, encoding="ISO-8859-1")


Processing 2021 ...


/var/folders/cr/048sbrkd4k5f7y4yw3rstw7m0000gn/T/ipykernel_52998/3014866597.py:22: DtypeWarning: Columns (7,12,15,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df_data = pd.read_csv(data_file, encoding="ISO-8859-1")


Processing 2022 ...


/var/folders/cr/048sbrkd4k5f7y4yw3rstw7m0000gn/T/ipykernel_52998/3014866597.py:22: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_data = pd.read_csv(data_file, encoding="ISO-8859-1")


Processing 2023 ...
Processing 2024 ...
Processing 2025 ...


In [ ]:
# Saving the 'vaers_all' as csv which contains data from all years

vaers_all.to_csv('/Users/joeraymond/Desktop/Data_Science_Projects/ADEGuard_AI_System/data/processed/processed_data/vaers_all.csv', index=False)

In [13]:
len(vaers_all)

1112050

### Exploratory data analysis of the VAERS dataset

1. Nan's

- Some 'SYMPTOM_TEXT' values are Nan, it has value _Unevaluable event_ in the column 'ALL_SYMPTOMS' These records can be removed.

In [ ]:
nan_count = vaers_all.isna().sum()
print(nan_count)

In [17]:
vaers_all['RECOVD'].value_counts()

N    358956
Y    332928
U    272124
Name: RECOVD, dtype: int64

In [18]:
vaers_all['DIED'].value_counts()

Y    24555
Name: DIED, dtype: int64

In [19]:
vaers_all['ER_VISIT'].value_counts()

Y    165
Name: ER_VISIT, dtype: int64

In [ ]:
sym_null = vaers_all[vaers_all['SYMPTOM_TEXT'].isna()]
sym_null

### Dropping rows that have Nan in 'SYMPTOM_TEXT' 

In [ ]:
vaers_all = vaers_all[vaers_all["SYMPTOM_TEXT"].notna()]

In [30]:
len(vaers_all)

1111040

In [32]:
vaers_all.isna().sum()

VAERS_ID                 0
RECVDATE                 0
STATE               184061
AGE_YRS             110474
CAGE_YR             223366
CAGE_MO            1105214
SEX                    255
RPT_DATE           1109845
SYMPTOM_TEXT             0
DIED               1086490
DATEDIED           1089031
L_THREAT           1091770
ER_VISIT           1110875
HOSPITAL            993885
HOSPDAYS           1042869
X_STAY             1110459
DISABLE            1087135
RECOVD              147635
VAX_DATE             79572
ONSET_DATE          105787
NUMDAYS             148758
LAB_DATA            677829
V_ADMINBY                0
V_FUNDBY           1109732
OTHER_MEDS          541899
CUR_ILL             664593
HISTORY             491889
PRIOR_VAX          1060988
SPLTTYPE            774848
FORM_VERS                0
TODAYS_DATE          15786
BIRTH_DEFECT       1110255
OFC_VISIT           884121
ER_ED_VISIT         973005
ALLERGIES           610992
ORDER                    0
VAX_TYPE                 0
V

In [33]:
vaers_all['RECOVD'].value_counts()

N    358902
Y    332629
U    271874
Name: RECOVD, dtype: int64

### Choosing only relevant columns from 'vaers_all' to remove noise from the dataset.

1. Columns 'VAERS_ID' and 'SYMPTOM_TEXT' are essential for annotating and uniquly identifying each record.

#### Filtering only 'VAERS_ID' and 'SYMPTOM_TEXT' and 'YEAR' from vaers_all and saving it to data directory for annotations.

In [ ]:
# Selecting only relevant columns from 'vaers_all' to remove noise from the dataset.

vaers_all_to_annotate = vaers_all[['VAERS_ID', 'SYMPTOM_TEXT', 'YEAR']]
vaers_all_to_annotate.to_csv('/Users/joeraymond/Desktop/Data_Science_Projects/ADEGuard_AI_System/data/processed/data_to_annotate/vaers_all_to_annotate.csv', index=False)